## Домашнее задание к занятию «Классификация: «Функции потерь и оптимизация»

### Задание

<b>Цель:</b><br> изучить применение методов оптимизации для решения задачи классификации<p>
<b>Описание задания:</b><br>
В домашнем задании необходимо применить полученные знания в теории оптимизации и машинном обучении для реализации логистической регрессии.<p>

<b>Этапы работы:</b><p>

1. Загрузите данные. Используйте <a href="https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html" target="_blank">датасет</a> с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только <b>2</b> класса: <u>Iris Versicolor, Iris Virginica.</u>
2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции. *Оформите в виде класса с методами.
3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.
4. Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).
5. Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam).
6. Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.<p>

Для лучшего понимания темы и упрощения реализации можете обратиться к <a href="https://habr.com/en/post/318970/" target="_blank">статье</a>.

Для получение зачета по этому домашнему заданию, минимально, должно быть реализовано обучение логистической регрессии и градиентный спуск.

<b>Результат:</b> получены навыки реализации методов оптимизации в задаче бинарной классификации. Пройденные методы оптимизации используются и в нейросетях.<p>
<b>Форма выполнения:</b> ссылка на Jupyter Notebook, загруженный на GitHub; ссылка на Google Colab; файл с расширением .ipynb.<p>
<b>Инструменты:</b> Jupyter Notebook/Google Colab; GitHub. <p>

<b><i>Рекомендации к выполнению: </b></i><br>
- Текст оформляйте в отдельной ячейке Jupyter Notebook/Google Colab в формате markdown.
- У графиков должен быть заголовок, подписи осей, легенда (опционально). Делайте графики бОльшего размера, чем стандартный вывод, чтобы увеличить читаемость.
- Убедитесь, что по ссылкам есть доступ на чтение/просмотр.
- Убедитесь, что все ячейки в работе выполнены и можно увидеть их вывод без повторного запуска.

### Решение:

> Импортируем библиотеки и датасет

In [16]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt

In [2]:
# Загружаем датасета iris
iris = load_iris()


X = iris.data
y = iris.target
X = iris.data[iris.target != 0] # убираем все данные для класса 0 из X
y = iris.target[iris.target != 0] # убираем все данные для класса 0 из Y
y[y==2] = 0 # Изменим метки классов на 0 и 1 (заменив 2 на 0)
X = np.insert(X, 0, 1, axis=1) # Добавим столбец с единицами для обучения theta0

In [3]:
# Создание DataFrame из датасета iris
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
df = df[df['target'] != 0]  # оставляем только Iris Versicolor и Iris Virginica


> Разделяем набор данных на наборы для обучения и тестирования

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

> Создаем класс LogisticRegression: реализация логистической регрессии методом градиентного спукса

In [5]:
class LogisticRegression:
    
    def __init__(self, learning_rate=0.01, num_iterations=3000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
    
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def fit(self, X, y):
        self.theta = np.zeros(X.shape[1])
        m = len(y)
        for i in range(self.num_iterations):
            z = np.dot(X, self.theta)
            h = self.sigmoid(z)
            gradient = np.dot(X.T, (h-y))/m
            self.theta -= self.learning_rate * gradient
    
    def predict(self, X):
        prob = self.sigmoid(np.dot(X, self.theta))
        return [1 if x >= 0.5 else 0 for x in prob]


> accuracy_score - метрика качетсва вручную и с помощью библиотеки

In [6]:
def accuracy(y_test, y_pred):
    return np.mean(y_test == y_pred)

In [7]:
start_time = time.time()
LR = LogisticRegression()
LR.fit(X_train, y_train)
y_pred1 = LR.predict(X_test)
acc1 = accuracy(y_test, y_pred1)
print('Accuracy:', acc1)
end_time = time.time()
execution_time1 = end_time - start_time

Accuracy: 0.9333333333333333


In [8]:
from sklearn.metrics import accuracy_score
accuracy1 = accuracy_score(y_test, y_pred1)
print('Accuracy:', accuracy1)

Accuracy: 0.9333333333333333


<b><i>Вывод 1</b> - Дали одинаковый результат, буду использовать библиотеку</i>

> Метод скользящего среднего (Root Mean Square Propagation, RMSProp) <a href="https://puzzlelib.org/ru/documentation/base/optimizers/RMSProp/#:~:text=RMSProp%20" target="_blank">Описание метода RMSProp </a>


In [9]:
class RMSProp:
    
    def __init__(self, learning_rate=0.01, beta=0.9, epsilon=1e-8, num_iterations=3000):
        self.learning_rate = learning_rate
        self.beta = beta
        self.epsilon = epsilon
        self.num_iterations = num_iterations
        
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def fit(self, X, y):
        self.theta = np.zeros(X.shape[1])
        v = np.zeros(X.shape[1])
        m = len(y)
        for i in range(self.num_iterations):
            z = np.dot(X, self.theta)
            h = self.sigmoid(z)
            gradient = np.dot(X.T, (h-y))/m
            v = self.beta * v + (1 - self.beta) * np.square(gradient)
            self.theta -= self.learning_rate * gradient / (np.sqrt(v) + self.epsilon)
    
    def predict(self, X):
        prob = self.sigmoid(np.dot(X, self.theta))
        return [1 if x >= 0.5 else 0 for x in prob]


In [10]:
start_time = time.time()
RMS = LogisticRegression()
RMS.fit(X_train, y_train)
y_pred2 = RMS.predict(X_test)
accuracy2 = accuracy_score(y_test, y_pred2)
print('Accuracy:', accuracy2)
end_time = time.time()
execution_time2 = end_time - start_time

Accuracy: 0.9333333333333333


> Метод адаптивной оценки моментов по Нестерову (Nesterov–accelerated Adaptive Moment Estimation, Nadam). <a href="https://proproprogs.ru/neural_network/optimizatory-v-keras-formirovanie-vyborki-validacii" target="_blank">Описание метода </a>

In [11]:
class Nadam:
    
    def __init__(self, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8, num_iterations=3000):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.num_iterations = num_iterations
        
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def fit(self, X, y):
        self.theta = np.zeros(X.shape[1])
        m_hat = np.zeros(X.shape[1]) # Используется для коррекции смещения на 1-ом шаге оптимизации
        v_hat = np.zeros(X.shape[1])
        m = len(y)
        for i in range(self.num_iterations):
            z = np.dot(X, self.theta)
            h = self.sigmoid(z)
            gradient = np.dot(X.T, (h-y))/m
            m_hat = self.beta1 * m_hat + (1 - self.beta1) * gradient
            v_hat = self.beta2 * v_hat + (1 - self.beta2) * np.square(gradient)
            m_hat_corr = m_hat / (1 - np.power(self.beta1, i+1))
            v_hat_corr = v_hat / (1 - np.power(self.beta2, i+1))
            self.theta -= self.learning_rate * m_hat_corr / (np.sqrt(v_hat_corr) + self.epsilon)
    
    def predict(self, X):
        prob = self.sigmoid(np.dot(X, self.theta))
        return [1 if x >= 0.5 else 0 for x in prob]

In [12]:
start_time = time.time()
Nad = Nadam()
Nad.fit(X_train, y_train)
y_pred3 = Nad.predict(X_test)
accuracy3 = accuracy_score(y_test, y_pred3)
print('Accuracy:', accuracy3)
end_time = time.time()
execution_time3 = end_time - start_time

Accuracy: 0.9


In [14]:
print('| Метод оптимизации | Точность | время выполнения  |')
print('|-------------------|----------|')
print(f'| Градиентный спуск | {accuracy1:.4f}   |  {execution_time1:.4f}  |')
print(f'| RMSProp           | {accuracy2:.4f}   |  {execution_time2:.4f}  |')
print(f'| Nadam             | {accuracy3:.4f}   |  {execution_time3:.4f}  |')

| Метод оптимизации | Точность | время выполнения  |
|-------------------|----------|
| Градиентный спуск | 0.9333   |  0.0431  |
| RMSProp           | 0.9333   |  0.0505  |
| Nadam             | 0.9000   |  0.0826  |


> <b>Вывод:</b> <br>Из таблицы видно, что Градиентый спуск работает быстрее, RMSProp и Градиентый спуск более точныею       